In [2]:
import os
import json
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, validation_curve
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import explained_variance_score, r2_score, mean_squared_error

In [6]:
df_reg = pd.read_csv(os.path.join(os.getcwd(), 'data', 'working', 'AHS Household Reg.csv'))
df_class =pd.read_csv(os.path.join(os.getcwd(), 'data', 'working', 'AHS Household Class.csv'))
y = df['RATINGHS'].copy().values
X = df.drop(['RATINGHS','CONTROL','YEAR','FIRSTHOME'], axis=1).copy().values

In [7]:
df_reg.shape

(27934, 331)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size=0.66)

C:\Users\Michael\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [155]:
classifiers = [LinearRegression(),
              Lasso(),
              Ridge()]

In [166]:
for classifier in classifiers:
    pipe = Pipeline(steps=[('classifier', classifier)])
    pipe.fit(X_train, y_train)
    print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test))
    pipe.
    print('\n')

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)
model score: 0.116


Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)
model score: 0.028


Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)
model score: 0.123




Trying to make polynomials work

In [157]:
def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), 
                         LinearRegression(**kwargs))

In [162]:
poly_model = make_pipeline(PolynomialFeatures(2), LinearRegression())

In [163]:
poly_model.fit(X_train, y_train)

MemoryError: 

In [ ]:
y_model = poly_model.predict(X_test[:, np.newaxis], y)

In [94]:
def fit_and_evaluate(dataset, model, label, **kwargs):
    start  = time.time() # Start the clock! 
    scores = {'precision':[], 'recall':[], 'accuracy':[], 'f1':[]}
    
    kf = KFold(n_splits = 12, shuffle=True)
    
    for train, test in kf.split(dataset.data):
        X_train, X_test = dataset.data[train], dataset.data[test]
        y_train, y_test = dataset.target[train], dataset.target[test]
        
        estimator = model(**kwargs)
        estimator.fit(X_train, y_train)
        
        expected  = y_test
        predicted = estimator.predict(X_test)
        
        # Append our scores to the tracker
        scores['precision'].append(metrics.precision_score(expected, predicted, average="weighted"))
        scores['recall'].append(metrics.recall_score(expected, predicted, average="weighted"))
        scores['accuracy'].append(metrics.accuracy_score(expected, predicted))
        scores['f1'].append(metrics.f1_score(expected, predicted, average="weighted"))

    # Report
    print("Build and Validation of {} took {:0.3f} seconds".format(label, time.time()-start))
    print("Validation scores are as follows:\n")
    print(pd.DataFrame(scores).mean())
    
    # Write official estimator to disk
    estimator = model(**kwargs)
    estimator.fit(dataset.data, dataset.target)
    
    outpath = label.lower().replace(" ", "-") + ".pickle"
    with open(outpath, 'wb') as f:
        pickle.dump(estimator, f)

    print("\nFitted model written to:\n{}".format(os.path.abspath(outpath)))

In [110]:
poly = PolynomialFeatures(3, include_bias=False)
poly.fit_transform(X_train)

MemoryError: 

Just do the models...

In [173]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print('R-Squared: {}'.format(r2_score(y_test, y_pred)))

R-Squared: 0.11593292920423126
